<a href="https://colab.research.google.com/github/paquino11/plots/blob/main/SadTalker_Beta_Monster_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

  #### SadTalker API allows you to create realistic talking face animations that sync seamlessly with audio input.

  #### Developed by [Tencent AI Lab, Xi'an Jiaotong University and Ant Lab](https://github.com/OpenTalker/SadTalker), SadTalker enhances talking head video realism by using 3D motion coefficients learned from audio.

### Set MonsterAPI Key

If you don't have a key then sign up on [MonsterAPI](https://monsterapi.ai/signup) to get a free key today.

Copy and Paste your Key below:

In [ ]:
# Set your Monster API key as an environment variable
MONSTER_API_KEY = 'YOUR_MONSTERAPI_KEY'

### Install Dependencies

In [ ]:
!apt-get install curl
from IPython.display import HTML
from base64 import b64encode
import subprocess
import json
import requests

def play_video_from_json(json_data):
    try:
        # Extract the video URL from the JSON data
        video_url = json_data["result"]["output"][0]
        # Generate HTML to embed a video player
        video_html = f'<video width="640" height="360" controls><source src="{video_url}" type="video/mp4"></video>'
        # Display the video player
        display(HTML(video_html))
    except KeyError:
        print("Invalid JSON format. Missing required keys.")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
curl is already the newest version (7.81.0-1ubuntu1.14).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


## Input Parameters -
**audio_url** [REQUIRED]
- *Type*: string
- *Format*: binary
- *Description*: Input audio or video file. Supported file formats: `mp3, mp4, wav, webm, ogg`

**image_url** [REQUIRED]
- *Type*: string
- *Format*: binary
- *Description*: Input Image File. Supported Formats: `.jpg, .jpeg, .png`

**enhancer**
- *Type*: string
- *Default*: gfpgan
- *Description*: Using 'gfpgan' or 'RestoreFormer' to enhance the generated face via face restoration network.

**preprocess**
- *Type*: string
- *Default*: full
- *Description*: Run and produce the results in the cropped input image. Other choices include resize, where the images will be resized to the specific resolution.

Options Include = [
    "crop",
    "extcrop",
    "resize",
    "full",
    "extfull"
]


**expression_scale**
- *Type*: number
- *Format*: float
- *Default*: 1.0
- *Minimum*: 0.0
- *Maximum*: 1.5
- *Description*: Adjust the strength of expression motion. A higher value makes the facial expressions more pronounced.

**still**
- *Type*: boolean
- *Default*: true
- *Description*: Set this option to 'true' to retain the same pose parameters as the original image, resulting in less head motion in the generated animation.


### Send POST Request

In [ ]:
# Set the URL and headers
url = "https://api.monsterapi.ai/v1/generate/sadtalker-beta"

headers = {
  'Content-Type': 'application/json',
  "Authorization": f"Bearer {MONSTER_API_KEY}"
}

# Set the data payload
payload = json.dumps({
    "audio_url": "https://audio-samples.github.io/samples/mp3/blizzard_primed/sample-0.mp3",
    "image_url": "https://processed-model-result.s3.us-east-2.amazonaws.com/173bdd0a-d38e-41b0-b141-755451549e52_1.png",
    "expression_scale": 1.3
})

# Send the POST request
response = requests.post(url, headers=headers, data=payload)

# Check if the response status code is 200 (OK)
if response.status_code == 200:
    response_data = response.json()
    process_id = response_data.get('process_id')
    print("Request Accepted Successfully!")
    print(f"Process ID: {process_id}")
else:
    print(f"Error: Status Code {response.status_code}")

Request Accepted Successfully!
Process ID: 0bc7122b-61e6-4431-b3a1-b6537b24a211


### Wait and GET Result

##### This step may take 5-7 minutes depending on audio length. On average it takes 9-15 seconds to produce 1 second of video clip.

In [ ]:
import requests
import time  # Import the time module

url = f"https://api.monsterapi.ai/v1/status/{process_id}"

headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {MONSTER_API_KEY}"
}

while True:
    response = requests.get(url, headers=headers)
    data = response.json()  # Parse the JSON response
    # print(data)
    if data['status'] == 'COMPLETED':
        play_video_from_json(data)
        break
    elif data['status'] == 'FAILED':
        print("Processing failed")
        break
    time.sleep(1)  # Wait for 1 seconds before the next API request (adjust this as needed)